# Pruebas para verificar el cumplimiento del objetivo A

## Configuración Inicial y Autenticación

In [1]:
from utils import configure_headers


# configure the headers
configure_headers()

ENDPOINTS_TO_TEST = [
    # search
    "https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album",
    # get album
    "https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy",
    # get artist
    "https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg",
    # get playlist
    "https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n",
    # get track
    "https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl",
    # TODO Puedes añadir más endpoints aquí
]

✅ Token loaded.


# Objetivo A - Prueba base

Hacemos las pruebas base para cumplir con el objetivo A. Vamos a probar todos los endpoints que declaramos que son los servicios a estudiar, y vamos a tomar todas las mediciones para luego poder responder a las metricas establecidas.

In [2]:
from utils import simulate_concurrent_users


simulate_concurrent_users(
    num_users=30,
    num_requests_per_user=30,
    endpoints_to_test=ENDPOINTS_TO_TEST
)


--- Simulation started with 30 concurrent users ---
Each user will do 30 requests.

✴️ Usuario 1 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1748095141.8350122
✴️ Usuario 2 - Starting requests to https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy. - Test started at 1748095141.8791375
✴️ Usuario 3 - Starting requests to https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg. - Test started at 1748095141.9789646
✴️ Usuario 4 - Starting requests to https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n. - Test started at 1748095142.0485284
✴️ Usuario 5 - Starting requests to https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl. - Test started at 1748095142.090399
✴️ Usuario 6 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1748095142.1818187
✴️ Usuario 7 - Starting requests to 

## Procesa los resultados

In [3]:
from utils import process_concurrent_results

# process the results
df_global_results, df_individual_results, df_by_endpoint = process_concurrent_results()


--- Resumen General de la Simulación Concurrente ---
Total de usuarios simulados: 30
Peticiones totales realizadas: 900
Peticiones exitosas totales: 406
Errores 4xx totales: 494
Errores 5xx totales: 0
Tasa de éxito global: 45.11%
Tasa de error 4xx global: 54.89%
Tasa de error 5xx global: 0.00%
Latencia promedio: 0.235


In [4]:
print('\nGlobal results\n')
df_global_results


Global results



,user_id,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,total_response_time,latencies,errors_occurred,test_duration
0,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,15,15,0,5.267505,"[0.17192673683166504, 0.2252364158630371, 0.18...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",35.286432
1,4,https://api.spotify.com/v1/playlists/3cEYpjA9o...,30,13,17,0,5.823115,"[0.20561766624450684, 0.2821042537689209, 0.21...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",35.846239
2,10,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,15,15,0,5.399158,"[0.19052457809448242, 0.213608980178833, 0.190...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",35.418922
3,20,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,14,16,0,5.028703,"[0.19655060768127441, 0.1734786033630371, 0.19...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",35.050400
4,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,14,16,0,6.443600,"[0.1812915802001953, 0.18222665786743164, 0.19...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",36.466231
5,7,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,13,17,0,6.168485,"[0.25717949867248535, 0.24216008186340332, 0.2...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",36.188683
6,1,https://api.spotify.com/v1/search?q=remaster%2...,30,14,16,0,6.896776,"[0.3203861713409424, 0.349285364151001, 0.2943...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",36.913995
7,9,https://api.spotify.com/v1/playlists/3cEYpjA9o...,30,15,15,0,6.413349,"[0.3061659336090088, 0.23144865036010742, 0.22...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",36.434368
8,18,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,14,16,0,6.189900,"[0.22028493881225586, 0.19120073318481445, 0.1...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",36.208660
9,29,https://api.spotify.com/v1/playlists/3cEYpjA9o...,30,14,16,0,5.740639,"[0.2131493091583252, 0.22980499267578125, 0.21...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",35.761868


In [5]:
print('\nIndividual results\n')
df_individual_results


Individual results



,user_ids,endpoints,status_codes,latencies,times,error_intervals
0,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.171927,1.748095e+09,NaN
1,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.225236,1.748095e+09,NaN
2,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.187235,1.748095e+09,NaN
3,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,200,0.205574,1.748095e+09,NaN
4,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,429,0.148461,1.748095e+09,NaN
...,...,...,...,...,...,...
895,14,https://api.spotify.com/v1/playlists/3cEYpjA9o...,429,0.160929,1.748095e+09,5.490870
896,14,https://api.spotify.com/v1/playlists/3cEYpjA9o...,429,0.152445,1.748095e+09,1.153703
897,14,https://api.spotify.com/v1/playlists/3cEYpjA9o...,429,0.154570,1.748095e+09,1.155297
898,14,https://api.spotify.com/v1/playlists/3cEYpjA9o...,429,0.158616,1.748095e+09,1.160046


In [6]:
print('\nResults by endpoint\n')
df_by_endpoint


Results by endpoint



,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,avg_latency,num_users,success_rate,error_4xx_rate,error_5xx_rate
0,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,180,81,99,0,0.248297,6,45.000000,55.000000,0.0
1,https://api.spotify.com/v1/artists/0TnOYISbd1X...,180,80,100,0,0.236927,6,44.444444,55.555556,0.0
2,https://api.spotify.com/v1/playlists/3cEYpjA9o...,180,80,100,0,0.232803,6,44.444444,55.555556,0.0
3,https://api.spotify.com/v1/search?q=remaster%2...,180,81,99,0,0.247784,6,45.000000,55.000000,0.0
4,https://api.spotify.com/v1/tracks/11dFghVXANMl...,180,84,96,0,0.209354,6,46.666667,53.333333,0.0


## Guardamos los resultados base

In [7]:
df_global_results.to_csv('./files/obj-A/global_results.csv', index=False)
df_individual_results.to_csv('./files/obj-A/individual_results.csv', index=False)
df_by_endpoint.to_csv('./files/obj-A/endpoints_results.csv', index=False)